In [1]:
import cv2
import numpy as np
from scipy.interpolate import CubicSpline
from skimage.restoration import denoise_tv_chambolle
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

%run "Read_image.py"

def print_dataset_header(dataset_name):
    total_length = 107 ## here need to be odd
    half_length = (total_length - len(dataset_name) - 11) // 2
    header = '#' * total_length
    dataset_header = f"{header}\n{'#' * half_length} Dataset: {dataset_name} {'#' * (half_length + len(dataset_name) % 2)}\n{header}"
    print(dataset_header)


from IPython.display import display
import pandas as pd

def printData(df):
    return display(pd.DataFrame(df))





In [2]:
image_Normal_ = ReadData(path = '.\\Dataset\\NORMAL' , extension = '.jpg')
image_DME_ = ReadData(path = '.\\Dataset\\DME' , extension = '.jpg')

In [3]:
largest_area = 0
shape_image = None

for img_normal, img_dme in zip(image_Normal_, image_DME_):
    height_normal, width_normal, _ = img_normal.shape
    height_dme,    width_dme,    _ = img_dme.shape
    hypotenuse_normal = np.sqrt(height_normal**2 + width_normal**2)
    hypotenuse_dme    = np.sqrt(height_dme**2    + width_dme**2)
    if hypotenuse_normal > hypotenuse_dme:
        if hypotenuse_normal >= largest_area: 
            largest_area = hypotenuse_normal * 1
            shape_image = (width_normal, height_normal)
    else:
        if hypotenuse_dme >= largest_area:
            largest_area = hypotenuse_dme * 1
            shape_image = (width_dme, height_dme)


image_Normal = []
image_DME = []

for img_normal, img_dme in zip(image_Normal_, image_DME_):
    img_dme = cv2.resize(img_dme, shape_image, interpolation =cv2.INTER_CUBIC)
    img_normal = cv2.resize(img_normal, shape_image, interpolation =cv2.INTER_CUBIC)
    image_Normal.append(img_normal)
    image_DME.append(img_dme)



In [4]:

for i, image in enumerate(image_Normal):
    img = np.float32(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    img = denoise_tv_chambolle(img, weight= 80) # it take a litle time 

    mask = np.zeros_like(img)
    perimeter = np.zeros_like(img)

    # cv2.getGaborKernel(ksize, sigma, theta, lambda, gamma, psi, ktype)
    ksize_ = 20 # - size of gabor filter (n, n)
    sigma_ = 12 # - standard deviation of the gaussian function
    theta_ = np.radians(95) # range(0, 180, 45) - orientation of the normal to the parallel stripes
    lambda_ = 20 # np.radians(135) # range(5, 15, 5) - wavelength of the sunusoidal factor
    gamma_ = 1 # range(0.5, 1, 0.1) - spatial aspect ratio
    psi_ =  np.radians(135) # range(0, 180, 45) - phase offset
    # ktype_ = # - type and range of values that each pixel in the gabor kernel can hold

    g_kernel = cv2.getGaborKernel((ksize_, ksize_), sigma_, theta_, lambda_, gamma_, psi_, ktype=cv2.CV_32F)

    filtered_img = cv2.filter2D(img, cv2.CV_8UC3, g_kernel)
    _, binary_img = cv2.threshold(filtered_img, 250, 1, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    final_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 10000:
            final_contours.append(contour)

    filtered_contours = []
    for contour in final_contours:
        points = contour[:, 0, :]  # Extracting the contour points
        row_max = np.argmax(points[:, 0])
        row_min = np.argmin(points[:, 0]) 
        row_min = np.where(points[:, 0] == points[row_min, 0] )
        row = np.arange(np.max(row_min), np.min(row_max) )
        points = points[row, :]
        filtered_contours.append(points)

    # ### Second countours
    np.random.seed(123)
    n_second = filtered_contours[1].shape[0]
    indices_first = sorted(np.random.choice(n_second, size=int(n_second * 0.7), replace=False))
    point_first = filtered_contours[1][indices_first , :]
    unique_values, unique_indices = np.unique(point_first[:, 0], return_index=True)
    point_first = point_first[unique_indices]
    x, y = point_first[:, 0], point_first[:, 1]
    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    y_sorted = y[sorted_indices]
    splines_coefficients =  CubicSpline(x_sorted, y_sorted, bc_type='natural')
    new_x = np.arange(np.min(x_sorted), np.max(x_sorted), 1)
    new_y = splines_coefficients(new_x)

    coord1 = np.stack((new_x, new_y - 5), axis=1).astype(np.int64)
    coord1 = coord1[np.argsort(coord1[:, 0], axis=None)]

    # First countours
    np.random.seed(123)
    n_first = filtered_contours[0].shape[0]
    indices_first = sorted(np.random.choice(n_first, size=int(n_first * 0.7), replace=False))
    point_first = filtered_contours[0][indices_first , :]
    unique_values, unique_indices = np.unique(point_first[:, 0], return_index=True)
    point_first = point_first[unique_indices]
    x, y = point_first[:, 0], point_first[:, 1]
    poly_coefficients = np.polyfit(x, y, 5)
    new_y =np.polyval(poly_coefficients, new_x )

    coord2 = np.stack((new_x, new_y + 5), axis=1).astype(np.int64)
    coord2 = coord2[np.argsort(-coord2[:, 0], axis=None)]

    coordinates = np.concatenate((coord1, coord2), axis=0)
    coordinates = np.expand_dims(coordinates, axis=1)

    folder_path3 = ".\\Dataset\\NORMAL_reshape"
    cv2.imwrite(f"{folder_path3}/{i:03d}.jpg", image)

    cv2.drawContours(image, [coordinates], -1, (255, 0, 0), 2)
    cv2.drawContours(mask, [coordinates],  -1, (255, 255, 255), -2) 
    cv2.drawContours(perimeter, [coordinates],  -1, (255, 255, 255), 2)

    folder_path0 = ".\\Dataset\\test_NORMAL"
    folder_path1 = ".\\Dataset\\NORMAL_mask"
    folder_path2 = ".\\Dataset\\NORMAL_perimeter"
    
    cv2.imwrite(f"{folder_path0}/{i:03d}_new_image.jpg", image)
    cv2.imwrite(f"{folder_path1}/{i:03d}_mask.jpg", mask)
    cv2.imwrite(f"{folder_path2}/{i:03d}_perimeter.jpg", perimeter)
  

In [5]:

for i, image in enumerate(image_DME):
    # print(i, end= ' ')

    img = np.float32(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    img = denoise_tv_chambolle(img, weight= 80) * 6 + 300

    mask = np.zeros_like(img)
    perimeter = np.zeros_like(img)

    # cv2.getGaborKernel(ksize, sigma, theta, lambda, gamma, psi, ktype)
    ksize_ = 20 # - size of gabor filter (n, n)
    sigma_ = 12 # - standard deviation of the gaussian function
    theta_ = np.radians(95) # range(0, 180, 45) - orientation of the normal to the parallel stripes
    lambda_ = 20 # range(5, 15, 5) - wavelength of the sunusoidal factor
    gamma_ = 1 # range(0.5, 1, 0.1) - spatial aspect ratio
    psi_ =  np.radians(140) # range(0, 180, 45) - phase offset
    # ktype_ = # - type and range of values that each pixel in the gabor kernel can hold

    g_kernel = cv2.getGaborKernel((ksize_, ksize_), sigma_, theta_, lambda_, gamma_, psi_, ktype=cv2.CV_32F)

    filtered_img = cv2.filter2D(img, cv2.CV_8UC3, g_kernel)
    _, binary_img = cv2.threshold(filtered_img, 250, 1, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    final_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 10000:
            final_contours.append(contour)

    filtered_contours = []
    for contour in final_contours:
        points = contour[:, 0, :]  # Extracting the contour points
        row_max = np.argmax(points[:, 0])
        row_min = np.argmin(points[:, 0]) 
        row_min = np.where(points[:, 0] == points[row_min, 0] )
        row = np.arange(np.max(row_min), np.min(row_max) )
        points = points[row, :]
        filtered_contours.append(points)

    # ### Second countours
    np.random.seed(123)
    n_second = filtered_contours[1].shape[0]
    indices_first = sorted(np.random.choice(n_second, size=int(n_second * 0.7), replace=False))
    point_first = filtered_contours[1][indices_first , :]
    unique_values, unique_indices = np.unique(point_first[:, 0], return_index=True)
    point_first = point_first[unique_indices]
    x, y = point_first[:, 0], point_first[:, 1]
    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    y_sorted = y[sorted_indices]
    splines_coefficients =  CubicSpline(x_sorted, y_sorted, bc_type='natural')
    new_x = np.arange(np.min(x_sorted), np.max(x_sorted), 1)
    new_y = splines_coefficients(new_x)

    coord1 = np.stack((new_x, new_y - 5), axis=1).astype(np.int64)
    coord1 = coord1[np.argsort(coord1[:, 0], axis=None)]

    # First countours
    np.random.seed(123)
    n_first = filtered_contours[0].shape[0]
    indices_first = sorted(np.random.choice(n_first, size=int(n_first * 0.7), replace=False))
    point_first = filtered_contours[0][indices_first , :]
    unique_values, unique_indices = np.unique(point_first[:, 0], return_index=True)
    point_first = point_first[unique_indices]
    x, y = point_first[:, 0], point_first[:, 1]
    poly_coefficients = np.polyfit(x, y, 5)
    new_y =np.polyval(poly_coefficients, new_x )

    coord2 = np.stack((new_x, new_y + 5), axis=1).astype(np.int64)
    coord2 = coord2[np.argsort(-coord2[:, 0], axis=None)]

    coordinates = np.concatenate((coord1, coord2), axis=0)
    coordinates = np.expand_dims(coordinates, axis=1)

    folder_path3 = ".\\Dataset\\DME_reshape\\"
    cv2.imwrite(f"{folder_path3}/{i:03d}.jpg", image)

    cv2.drawContours(image, [coordinates], -1, (255, 0, 0), 2)
    cv2.drawContours(mask, [coordinates],  -1, (255, 255, 255), -2) 
    cv2.drawContours(perimeter, [coordinates],  -1, (255, 255, 255), 2)

    folder_path0 = ".\\Dataset\\test_DME"
    folder_path1 = ".\\Dataset\\DME_mask\\"
    folder_path2 = ".\\Dataset\\DME_perimeter\\"
    
    cv2.imwrite(f"{folder_path0}/{i:03d}_new_image.jpg", image)
    cv2.imwrite(f"{folder_path1}/{i:03d}_mask.jpg", mask)
    cv2.imwrite(f"{folder_path2}/{i:03d}_perimeter.jpg", perimeter)


